### otx 패키지 다운로드

In [ ]:
!git clone https://github.com/openvinotoolkit/training_extensions.git

In [ ]:
%cd training_extensions

### OTX 설치

In [ ]:
!pip install  otx[full]

In [ ]:
!pip install mmcv-full==1.7.1

### Classification

In [ ]:
%cd /content

In [ ]:
!mkdir data

In [ ]:
import os
import tensorflow as tf
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

In [ ]:
!mv /root/.keras/datasets/cats_and_dogs_filtered/train /content/data/cats_and_dogs

In [ ]:
!ls /content/data/cats_and_dogs

In [ ]:
!otx find --template --task CLASSIFICATION

In [ ]:
!otx build --train-data-roots /content/data/cats_and_dogs --model MobileNet-V3-large-1x

In [ ]:
%cd otx-workspace-CLASSIFICATION

In [ ]:
!otx train

In [ ]:
!otx export --load-weights outputs/latest_trained_model/models/weights.pth --output openvino_model

In [ ]:
!otx deploy template.yaml --load-weights openvino_model/openvino.xml --output outputs/deploy

In [ ]:
!cp outputs/deploy/openvino.zip ..

In [ ]:
%cd ..

In [ ]:
!unzip openvino.zip

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from openvino.runtime import Core
from pathlib import Path

filename="/content/data/cats_and_dogs/cats/cat.51.jpg"
model_xml_path = 'model/model.xml'
model_bin_path = 'model/model.bin'
ie = Core()
model = ie.read_model(model=model_xml_path)
compiled_model = ie.compile_model(model=model, device_name="CPU")
output_layer = compiled_model.output(0)
image = cv2.imread(filename=filename)
resized_image = cv2.resize(image, (224, 224))
input_image = np.expand_dims(resized_image.transpose(2, 0, 1), 0)
result_infer = compiled_model([input_image])[output_layer]
result_index = np.argmax(result_infer)

input_image = np.expand_dims(input_image, 0)
plt.imshow(cv2.cvtColor(image, code=cv2.COLOR_BGR2RGB))
label_name=['cat', 'dog']
target_str = label_name[result_index]
plt.title(label_name[result_index])
plt.show()

### Object Detection

In [ ]:
!otx find --template --task DETECTION

In [ ]:
%cd /content/data

In [ ]:
!git clone https://github.com/thsant/wgisd.git

In [ ]:
%cd /content/data/wgisd
!mv data images
!mv coco_annotations annotations
!mv annotations/test_bbox_instances.json annotations/instances_val.json
!mv annotations/train_bbox_instances.json annotations/instances_train.json

In [ ]:
%cd /content

In [ ]:
!otx build --train-data-roots /content/data/wgisd --model MobileNetV2-ATSS

In [ ]:
%cd /content/otx-workspace-DETECTION/

In [ ]:
!otx train

In [ ]:
!otx export --load-weights outputs/latest_trained_model/models/weights.pth --output openvino_model

In [ ]:
!otx deploy template.yaml --load-weights openvino_model/openvino.xml --output outputs/deploy

In [ ]:
%cd /content/otx-workspace-DETECTION

In [ ]:
!rm /content/openvino.zip

In [ ]:
!cp outputs/deploy/openvino.zip ..

In [ ]:
%cd ..

In [ ]:
!unzip openvino.zip -d test

In [ ]:
%cd test

In [ ]:
!ls

In [ ]:
from re import X
from cv2.gapi import YUV2RGB
import cv2
import matplotlib.pyplot as plt
import numpy as np
from openvino.runtime import Core
from pathlib import Path

filename="/content/data/wgisd/images/CDY_2017.jpg"
model_xml_path = 'model/model.xml'
model_bin_path = 'model/model.bin'
ie = Core()
model = ie.read_model(model=model_xml_path)
compiled_model = ie.compile_model(model=model, device_name="CPU")
output_layer = compiled_model.output(0)
image = cv2.imread(filename=filename)
print(image.shape)
resized_image = cv2.resize(image, (992, 736))
# resized_image = cv2.resize(image, (224,224))
input_image = np.expand_dims(resized_image.transpose(2, 0, 1), 0)
result_infer = compiled_model([input_image])[output_layer]
# print(result_infer.shape)
# print(result_infer[0,0:4])
x_ratio = 992/image.shape[1]
y_ratio = 736/image.shape[0]

clone = image.copy()
for info in result_infer[0]:
    # print(info)

    x1= int(info[0]//x_ratio)
    y1= int(info[1]//y_ratio)
    x2= int(info[2]//x_ratio)
    y2= int(info[3]//y_ratio)
    conf = info[4]
    # print(x1,y1,x2,y2,conf)

    if conf > 0.3:
        cv2.rectangle(clone, (x1, y1), (x2, y2), (255, 0, 0), 2)


# print(clone.shape)
plt.imshow(cv2.cvtColor(clone, code=cv2.COLOR_BGR2RGB))

plt.show()